## Задача 3-2. Задача TSP: нижняя оценка Гельда—Карпа.

В этой задаче Вам предлагается релизовать алгоритм Гельда—Карпа для нижней оценки стоимости решения в задаче Euclidean TSP.

Сделайте следующее:
* Скачайте файл [`tsp-instances.zip`](https://github.com/dainiak/discrete-optimization-course/raw/master/tsp-instances.zip) и разархивируйте из него файлы со входами задачи TSP. Это в точности те же входные данные, что и в задании 3-1.
* Реализуйте функцию `lower_bound_tsp`. При этом можно пользоваться каким-нибудь стандартным алгоритмом построения минимального остовного дерева из библиотеки [`networkx`](https://networkx.github.io/), входящей в состав дистрибутива Anaconda.
* Запустите функцию `run_all()`, чтобы протестировать свой код, и напишите полученные, как следствия, верхние оценки погрешностей решений, которые были получены Вашими алгоритмами NN и NI при решении задания 3-1. Запишите свои выводы в 1-2 предложениях в последней ячейке ipynb-файла.

**Импортируем требуемые модули:**

In [91]:
import networkx
import numpy as np
import pandas
from typing import List, Tuple
from math import sqrt
from itertools import combinations, islice
import time
from os.path import exists

**Чтение графа и вычисление расстояния:**

In [105]:
def read_tsp_instance(filename: str) -> List[Tuple[int,int]]:
    with open(filename, 'r') as file:
        coordinates = []
        for line in file:
            line = line.strip().lower()
            if line.startswith('dimension'):
                coordinates = [(0, 0)] * int(line.split()[-1])
            tokens = line.split()
            if len(tokens) == 3 and tokens[0].isdecimal():
                tokens = line.split()
                coordinates[int(tokens[0])-1] = tuple(map(float, tokens[1:]))
        return coordinates


def euclidean_distance(point1: Tuple[int,int], point2: Tuple[int,int]) -> float:
    return sqrt((point1[0]-point2[0]) ** 2 + (point1[1]-point2[1]) ** 2)

**Создание графа:**

In [106]:
def graph(vertex_coordinates, coordinates):
    size = len(vertex_coordinates)
    g = networkx.Graph()
    g.add_nodes_from(range(size))
    for vertex, another_vertex in combinations(range(size), 2):
        g.add_edge(vertex, another_vertex, weight=euclidean_distance(vertex_coordinates[vertex], vertex_coordinates[another_vertex]) - coordinates[vertex] - coordinates[another_vertex])
    return g

**Обновление координат при новом MST:**

In [107]:
def boost(MST, coordinates):
    V = MST.number_of_nodes()
    boosted = np.zeros(V)
    for index, elem in enumerate(boosted):
        elem = coordinates[index] + 2 - MST.degree(index)
    return boosted

**Оценка Гельда-Карпа:**

In [118]:
def lower_bound_tsp(vertex_coordinates: List[Tuple[int,int]]) -> float:
    size = len(vertex_coordinates)
    coords = np.zeros(size)
    start, answer =  time.monotonic(), 0
    while time.monotonic() - start < 10:
        g = graph(vertex_coordinates, coords)
        MST = networkx.minimum_spanning_tree(g)
        current_answer = 2 * sum(coords) + MST.size(weight='weight')
        answer = max(answer, current_answer)
        coords = boost(MST, coords)
    return answer

**Запуск всех тестов:**

In [119]:
def run_all():
    NN, NI = 0, 0
    myNN = [47464, 85314, 18830, 131702, 44160, 62860, 92247]
    myNI = [52587, 88530, 17631, 130067, 39982, 57906, 87570]
    instance_filenames = ['pr107.tsp', 'pr152.tsp', 'd198.tsp', 
                          'pr439.tsp', 'd493.tsp', 'd657.tsp', 'd2103.tsp']
    for test, filename in enumerate(instance_filenames):
        if not exists(filename):
            print('File not found: “{}”. Skipping this instance.'.format(filename))
            continue
        instance = read_tsp_instance(filename)
        print('***********************')
        print('Тест {} из файла {}:'.format(test + 1, filename), end='\n')
        time_start = time.monotonic()
        estimation = lower_bound_tsp(instance)
        time_nn = time.monotonic()-time_start
        print(' Завершено за {:.2} секунд с нижней оценкой {}'.format(time_nn, int(estimation)))
        print('---------------------------')
        print(' Метод Nearest Neighbour давал результат {}\n Погрешность метода Nearest Neighbour равна {:.3}%'.format(myNN[test], (myNN[test] / estimation - 1) * 100))
        print('---------------------------')
        print(' Метод Nearest Insertion давал результат {}\n Погрешность метода Nearest Insertion равна {:.3}%'.format(myNI[test], (myNI[test] / estimation - 1) * 100))
        print('---------------------------')
        if myNN[test] < myNI[test]:
            print(' Метод Nearest Neighbour в данном тесте дал результат более близкий к нижней оценке Гельда-Карпа.\n')
            NN += 1
        else:
            print(' Метод Nearest Insertion в данном тесте дал результат более близкий к нижней оценке Гельда-Карпа.\n')
            NI += 1
        
    if NI >= NN:
        print('Метод Nearest Insertion оказался точнее в совокупности тестов.')
    else:
        print('Метод Nearest Neighbour оказался точнее в совокупности тестов.')

In [120]:
run_all()

***********************
Тест 1 из файла pr107.tsp:
 Завершено за 1e+01 секунд с нижней оценкой 34757
---------------------------
 Метод Nearest Neighbour давал результат 47464
 Погрешность метода Nearest Neighbour равна 36.6%
---------------------------
 Метод Nearest Insertion давал результат 52587
 Погрешность метода Nearest Insertion равна 51.3%
---------------------------
 Метод Nearest Neighbour в данном тесте дал результат более близкий к нижней оценке Гельда-Карпа.

***********************
Тест 2 из файла pr152.tsp:
 Завершено за 1e+01 секунд с нижней оценкой 59168
---------------------------
 Метод Nearest Neighbour давал результат 85314
 Погрешность метода Nearest Neighbour равна 44.2%
---------------------------
 Метод Nearest Insertion давал результат 88530
 Погрешность метода Nearest Insertion равна 49.6%
---------------------------
 Метод Nearest Neighbour в данном тесте дал результат более близкий к нижней оценке Гельда-Карпа.

***********************
Тест 3 из файла d198

## Выводы

В ходе работы мы получили нижние оценки для графов из примеров. Алгоритмы NN и NI дают вполне хорошие приближения к нижней оценке Гельда-Карпа. На нашем датасете точнее (ближе к нижней оценке) оказался метод NI.